In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from nltk.tokenize import TweetTokenizer
import re


# Find Hyperlinks in string

In [ ]:
def Find(string):
  
    # findall() has been used 
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)     
    return [x[0] for x in url]

# Data Loader / Tokenizer

In [20]:
train_data = pd.read_csv('train.csv')
tokenizer = TweetTokenizer()

tokens = tokenizer.tokenize(train_data['Tweet'][0])
output = []
for token in tokens:
    url = Find(token)
    if not url:
        output.append(token.lower())
    else:
        output.append('url')

print(train_data['Tweet'][0])
print(output)


It's the everything else that's complicated. #PESummit #PXpic.twitter.com/Jsv6BAFQMl
["it's", 'the', 'everything', 'else', "that's", 'complicated', '.', '#pesummit', '#pxpic', 'url']
